In [1]:
import numpy as np
from itertools import combinations
import fim
from scipy.spatial import distance
from scipy.optimize import linprog

In [2]:
def subset_of(trans_1, trans_2):
    ''' Check if trans_1 is subset of trans_2 '''
    for item in trans_1:
        if item not in trans_2:
            return False
    return True

In [3]:
def trans_support_dict(transactions):
    ''' Create a dictionary that stores transactions-supports pairs.
        key: transaction
        value: [og_support, 0]'''
    item_dict = {}
    for item in transactions:
        if item_dict.get(tuple(sorted(item))):
            item_dict[tuple(sorted(item))][0] += 1/len(transactions)
        else:
            item_dict[tuple(sorted(item))] = [1/len(transactions), 0]
    return item_dict        

In [4]:
def create_truth_table(items, clauses):
    ''' Given the items of the database and the restrictions/clauses of the problem
        create the truth table for the LP problem.'''
    powerset = 2**len(items)-1
    truth_table = np.zeros([len(clauses), powerset]).astype(np.int8)  
    l = 0
    for k in range(len(items)):
        combs = combinations(items, k+1)
        for i in range(l, powerset):
            try:
                _ = next(combs)
                for index, clause in enumerate(clauses):
                    if subset_of(clause[0], _):
                        truth_table[index, i] = 1
            except StopIteration as ex:
                l = i
                break
    truth_table = np.append(truth_table, np.ones(shape=[1, truth_table.shape[1]]), axis=0)        
    return truth_table

In [5]:
def reconstruct_db(items, reconstr_trans):
    ''' Reconstruct transaction database from the LP solution.
        Database is returned as a list of trans-support tuples.'''
    l = 0
    reconstr_db = []
    for k in range(len(reconstr_trans)):
        combs = combinations(items, k+1)
        for i in range(l, len(reconstr_trans)):
            try:
                _ = tuple(sorted(next(combs)))
                if reconstr_trans[i] != 0:
                    reconstr_db.append((_, reconstr_trans[i]))
            except StopIteration as ex:
                l = i
                break
    return reconstr_db

In [6]:
def get_negbord(support):
    freq_0 = fim.apriori(item_list, supp=0, target='s' , report='s')
    _ = list(filter(lambda x: x[1] < support/100, freq_0))
    a = sorted(_, key=lambda x: len(x[0]), reverse=True)
    negbor = []
    for i in range(len(a)-1):
        flag = True
        for j in range(i+1, len(a)):
            if subset_of(a[j][0], a[i][0]):
                flag = False
        if flag and len(a[i])==len(a[-1]):
            negbor.append(a[i])
    negbor.append(a[-1])
    return negbor

#### Import transactions from a file

In [7]:
with open(r'datasets\7_items\dataset_7_100000.csv', 'r') as f:
    items = []
    item_list = []
    for line in f:
        temp_line = line.strip()[:-1].replace('"', '').split(',')
        for item in temp_line:
            if item not in items:
                items.append(item)
        item_list.append(temp_line)
    items = set(items)

#### Create the transaction dictionary from the imported list of transactions
#### (or you can provide the transactions as a list) 

In [8]:
trans_dict = trans_support_dict(item_list)

#### Apply Apriori algorithm to calculate frequent itemsets

In [9]:
freq = fim.apriori(item_list, supp=30, target='s' , report='s')
closed = fim.apriori(item_list, supp=30, target='c' , report='s')
maximal = fim.apriori(item_list, supp=30, target='m' , report='s')
freq_nb = freq + get_negbord(30)
max_nb = maximal + get_negbord(30)

#### Create the truth table of the LP problem

In [10]:
A_eq = create_truth_table(items, freq)
# A_eq = create_truth_table(items, closed)
# A_eq = create_truth_table(items, maximal)

#### Formulate the LP problem (obj function, bounds...)

In [11]:
obj = np.ones(A_eq.shape[1])
b_eq = np.append(np.array([fr[1] for fr in freq]), np.array([1]), axis = 0)
bounds = (0, 1)

In [12]:
# obj = np.ones(A_eq.shape[1])
# b_eq = np.append(np.array([clos[1] for clos in closed]), np.array([1]), axis = 0)
# bounds = (0, 1)

In [13]:
# obj = np.ones(A_eq.shape[1])
# b_eq = np.append(np.array([maxim[1] for maxim in maximal]), np.array([1]), axis = 0)
# bounds = (0, 1)

In [14]:
# obj = np.ones(A_eq.shape[1])
# b_eq = np.append(np.array([frnb[1] for frnb in freq_nb]), np.array([1]), axis = 0)
# bounds = (0, 1)

#### Solve the problem 

In [15]:
a = linprog(c=obj, A_eq=A_eq, b_eq=b_eq, method='simplex', bounds=bounds)

#### Reconstruct the database from LP solution

In [16]:
recon_db = reconstruct_db(items, a.x)

#### Add reconstructed transactions supports to the trans dictionary

In [17]:
for item in recon_db:
    if item[0] in trans_dict.keys():
        trans_dict[item[0]][1] = abs(item[1])
    else:
        trans_dict[item[0]] = [0, abs(item[1])]

#### Calculate database distance

In [18]:
distance.jensenshannon([value[0] for value in trans_dict.values()], [value[1] for value in trans_dict.values()])

0.7800523645527374